In [2]:
import pandas as pd
import json
from datetime import datetime
from sqlalchemy import create_engine, types
import xmlrpc.client
import os
from dotenv import load_dotenv

load_dotenv()
DATABASE_CONNECTION_URI = os.environ["DB_URL"]
ODOO_URL = os.environ["ODOO_URL"]
ODOO_NAME = os.environ["ODOO_NAME"]
ODOO_USERNAME = os.environ["ODOO_USERNAME"]
ODOO_PASSWORD = os.environ["ODOO_PASSWORD"]

# create a connection to the database
engine = create_engine(DATABASE_CONNECTION_URI)

class OdooAPI:
    def __init__(self):
        self.dbUrl = ODOO_URL
        self.dbName = ODOO_NAME
        self.dbUsername = ODOO_USERNAME
        self.dbPassword = ODOO_PASSWORD
        self.common = xmlrpc.client.ServerProxy('{}/xmlrpc/2/common'.format(self.dbUrl))
        self.uid = self.common.authenticate(self.dbName, self.dbUsername, self.dbPassword, {})
        self.models = xmlrpc.client.ServerProxy('{}/xmlrpc/2/object'.format(self.dbUrl))
        self.country_codes = {
            "Argentina": "AR",
            "Paraguay": "PY",
            "Bolivia": "BO",
            "Uruguay": "UY",
            "Chile": "CL",
            "Perú": "PE",
            "Peru": "PE",
            "Ecuador": "EC",
            "Colombia": "CO",
            "Venezuela": "VE",
            "Brasil": "BR",
            "Panamá": "PA",
            "Panama": "PA",
            "Costa Rica": "CR",
            "Honduras": "HN",
            "El Salvador": "SV",
            "Guatemala": "GT",
            "México": "MX",
            "Mexico": "MX",
            "República Dominicana": "DM",
            "Republica Dominicana": "DM",
            "Rep Dom": "DM",
            "Puerto Rico": "PR",
            "España": "ES",
        }

    # Data Handling Functions
    def id_field_into_1value(self, jsonFile):
        for index, line in enumerate(jsonFile):
            for item in line:
                if type(line[item]) == list:
                    jsonFile[index][item] = jsonFile[index][item][1]

    # Export to CSV functions
    def raw_export_projects(self, properties_dict):
        projects = self.models.execute_kw(self.dbName,
                                          self.uid,
                                          self.dbPassword,
                                          'project.project',
                                          'search_read',
                                          [],
                                          {'fields': list(properties_dict.keys()), })
        self.id_field_into_1value(projects)
        df = pd.DataFrame(projects)

        # exporta los proyectos a un srchivo .csv
        df.to_csv("raw_odoo_projects.csv", encoding="latin-1", index=False, errors='ignore')

    def raw_export_subscriptions(self, properties_dict):
        subscriptions = self.models.execute_kw(self.dbName,
                                               self.uid,
                                               self.dbPassword,
                                               'sale.subscription.report',
                                               'search_read',
                                               [],
                                               {'fields': list(properties_dict.keys())})
        self.id_field_into_1value(subscriptions)
        df = pd.DataFrame(subscriptions)
        df.to_csv("raw_odoo_subscriptions.csv", encoding="latin-1", index=False, errors='ignore')
        #return subscriptions

    def raw_export_sales_lines(self, properties_dict):
        salesLines = self.models.execute_kw(self.dbName,
                                            self.uid,
                                            self.dbPassword,
                                            'sale.order.line',
                                            'search_read',
                                            [],
                                            {'fields': list(properties_dict.keys())})
        self.id_field_into_1value(salesLines)
        df = pd.DataFrame(salesLines)
        df.to_csv("raw_odoo_sales_lines.csv", encoding="latin-1", index=False, errors='ignore')

    def raw_export_entities(self, properties_dict):
        entities = self.models.execute_kw(self.dbName,
                                          self.uid,
                                          self.dbPassword,
                                          'res.partner',
                                          'search_read',
                                          [],
                                          {'fields': list(properties_dict.keys())})
        self.id_field_into_1value(entities)
        df = pd.DataFrame(entities)
        df.to_csv("raw_odoo_entities.csv", encoding="latin-1", index=False, errors='ignore')

    def raw_export_subscriptions_lines(self, properties_dict):
        subscriptions_lines = self.models.execute_kw(self.dbName,
                                          self.uid,
                                          self.dbPassword,
                                          'sale.subscription.line',
                                          'search_read',
                                          [],
                                          {'fields': list(properties_dict.keys())})
        self.id_field_into_1value(subscriptions_lines)
        df = pd.DataFrame(subscriptions_lines)
        df.to_csv("raw_odoo_subscriptions_lines.csv", encoding="latin-1", index=False, errors='ignore')

    def handle_raw_odoo(self, csv_file, properties_dict={}, values_dict={}, date_columns=[]):
        # Csv into Dataframe
        df = pd.read_csv(csv_file + ".csv", encoding="latin-1")
        # Rename Columns
        df.rename(columns=properties_dict, inplace=True)
        # Rename Values
        df = df.map(lambda x: str(x) if pd.notnull(x) else '')
        df.replace(values_dict, inplace=True)
        # Columns into lower case and underscore union
        columns = [column.lower() for column in df.columns]
        columns = ["_".join(column.split(" ")) for column in columns]
        df.columns = columns
        # Date Standarization
        date_columns = [char.lower() for char in date_columns]
        date_columns = ["_".join(char.split(" ")) for char in date_columns]
        for date_column in date_columns:
            df[date_column] = pd.to_datetime(df[df[date_column].notna()][date_column], errors='raise')
            df[date_column] = df[date_column].dt.strftime('%Y-%m-%d')
        df.to_csv(csv_file[4:] + ".csv", encoding="latin-1", index=False, errors='ignore')
        
class SqlAPI:

    def __init__(self):
        self.conn = create_engine(DATABASE_CONNECTION_URI)

    def str_type_into_sqltype(self, datatype_dict):
        for i in datatype_dict:
            if datatype_dict[i] == "string":
                datatype_dict[i] = types.TEXT()
            elif datatype_dict[i] == "int64":
                datatype_dict[i] = types.NUMERIC()
            elif datatype_dict[i] == "float64":
                datatype_dict[i] = types.FLOAT()
            elif datatype_dict[i] == "datetime64":
                datatype_dict[i] = types.TIMESTAMP()
            elif datatype_dict[i] == "bool":
                datatype_dict[i] = types.BOOLEAN()
        return datatype_dict

    def insert_df(self, dataframe, table, dtype, index=False, if_exists="replace"):
        dataframe.to_sql(
            name=table,
            con=self.conn,
            index=index,
            if_exists=if_exists,
            method='multi',
            dtype=dtype,
            chunksize=10000
        )

    def update_table(self, table, dtype):

        print("Update Started")
        csv_name = table + ".csv"
        df = pd.read_csv(csv_name, encoding='latin-1')
        columns = [column.lower() for column in df.columns]
        columns = ["_".join(column.split(" ")) for column in columns]
        df.columns = columns
        dtype = self.str_type_into_sqltype(dtype)
        table_name = table + "_test"
        self.insert_df(df, table_name, dtype)
        print("Update Finished " + table_name)

def dict_into_simple_dict(complex_dict, simple_value):
    headers = list(complex_dict.keys())
    simple_values_list = []
    for k in complex_dict:
        simple_values_list.append(complex_dict[k][simple_value])
    return dict(zip(headers, simple_values_list))


def list_of_datetimes(complex_dict):
    lst = []
    simple_dict = dict_into_simple_dict(complex_dict, "datatype")
    for item in simple_dict:
        if simple_dict[item] == "datetime64":
            lst.append(item)
    return lst

def update_database():
    # DB Schema Handling
    api_schema = json.load(open("api_schema.json"))["API"]
    odoo_schema = api_schema["Odoo"]

    # Odoo Schema
    o_subscriptions_properties = dict_into_simple_dict(odoo_schema["Subscriptions"]["Properties"], "header_name")
    o_subscriptions_values = odoo_schema["Subscriptions"]["Values"]
    o_subscriptions_datatypes = dict(zip(o_subscriptions_properties.values(),
                                         dict_into_simple_dict(odoo_schema["Subscriptions"]["Properties"],
                                                               "datatype").values()))
    o_subscriptions_datecolumns = list_of_datetimes(odoo_schema["Subscriptions"]["Properties"])
    o_subscriptions_datecolumns = [o_subscriptions_properties[i] for i in o_subscriptions_datecolumns]
    o_projects_properties = dict_into_simple_dict(odoo_schema["Projects"]["Properties"], "header_name")
    o_projects_values = odoo_schema["Projects"]["Values"]
    o_projects_datatypes = dict(zip(o_projects_properties.values(),
                                    dict_into_simple_dict(odoo_schema["Projects"]["Properties"], "datatype").values()))
    o_projects_datecolumns = list_of_datetimes(odoo_schema["Projects"]["Properties"])
    o_projects_datecolumns = [o_projects_properties[i] for i in o_projects_datecolumns]
    o_saleslines_properties = dict_into_simple_dict(odoo_schema["Sales Lines"]["Properties"], "header_name")
    o_saleslines_values = odoo_schema["Sales Lines"]["Values"]
    o_saleslines_datatypes = dict(zip(o_saleslines_properties.values(),
                                      dict_into_simple_dict(odoo_schema["Sales Lines"]["Properties"],
                                                            "datatype").values()))
    o_saleslines_datecolumns = list_of_datetimes(odoo_schema["Sales Lines"]["Properties"])
    o_saleslines_datecolumns = [o_saleslines_properties[i] for i in o_saleslines_datecolumns]
    o_entities_properties = dict_into_simple_dict(odoo_schema["Entities"]["Properties"], "header_name")
    o_entities_values = odoo_schema["Entities"]["Values"]
    o_entities_datatypes = dict(zip(o_saleslines_properties.values(),
                                    dict_into_simple_dict(odoo_schema["Entities"]["Properties"], "datatype").values()))
    o_entities_datecolumns = list_of_datetimes(odoo_schema["Entities"]["Properties"])
    o_entities_datecolumns = [o_entities_properties[i] for i in o_entities_datecolumns]
    o_subscriptionslines_properties = dict_into_simple_dict(odoo_schema["Subscriptions Lines"]["Properties"],
                                                            "header_name")
    o_subscriptionslines_values = odoo_schema["Subscriptions Lines"]["Values"]
    o_subscriptionslines_datatypes = dict(zip(o_subscriptionslines_properties.values(),
                                              dict_into_simple_dict(odoo_schema["Subscriptions Lines"]["Properties"],
                                                                    "datatype").values()))
    o_subscriptionslines_datecolumns = list_of_datetimes(odoo_schema["Subscriptions Lines"]["Properties"])
    o_subscriptionslines_datecolumns = [o_subscriptionslines_properties[i] for i in o_subscriptionslines_datecolumns]

    # Class Variables Declaration
    s = SqlAPI()
    o = OdooAPI()

    # Odoo Export
    o.raw_export_projects(o_projects_properties)
    o.raw_export_subscriptions(o_subscriptions_properties)
    o.raw_export_sales_lines(o_saleslines_properties)
    o.raw_export_entities(o_entities_properties)

    # Data Handle
    o.handle_raw_odoo("raw_odoo_projects", o_projects_properties, o_projects_values, o_projects_datecolumns)
    o.handle_raw_odoo("raw_odoo_subscriptions", o_subscriptions_properties, o_subscriptions_values,
                      o_subscriptions_datecolumns)
    o.handle_raw_odoo("raw_odoo_sales_lines", o_saleslines_properties, o_saleslines_values, o_saleslines_datecolumns)
    o.handle_raw_odoo("raw_odoo_entities", o_entities_properties, o_entities_values, o_entities_datecolumns)

    tables = [
        "odoo_projects",
        "odoo_subscriptions",
        "odoo_sales_lines",
        "odoo_entities",
    ]

    tables_datatypes = [
        o_projects_datatypes,
        o_subscriptions_datatypes,
        o_saleslines_datatypes,
        o_entities_datatypes,
    ]

    for index, table in enumerate(tables):
        s.update_table(table, tables_datatypes[index])

update_database()

# Delete the CSV file after writing
os.remove("raw_odoo_projects" + ".csv")
os.remove("raw_odoo_subscriptions" + ".csv")
os.remove("raw_odoo_sales_lines" + ".csv")
os.remove("raw_odoo_entities" + ".csv")
os.remove("odoo_projects" + ".csv")
os.remove("odoo_subscriptions" + ".csv")
os.remove("odoo_sales_lines" + ".csv")
os.remove("odoo_entities" + ".csv")



Update Started
Update Finished odoo_projects_test
Update Started
Update Finished odoo_subscriptions_test
Update Started
Update Finished odoo_sales_lines_test
Update Started
Update Finished odoo_entities_test
